In [2]:
import open3d as o3d 
import numpy as np
from pathlib import Path

In [3]:
BASE = Path().resolve().parents[0]
DATA_PATH = BASE / 'data'
print(DATA_PATH)
SHOW_PC = True

/home/dayoff/codes/point_cloud_lib/data


In [4]:
# get first point cloud
pc_path = DATA_PATH / 'kitti' / '0000000000.ply'

# supported types: pcd, ply, xyz, xyzrgb, xyzn, pts.
# I had to transform .txt files from kitti to ply
pcd = o3d.io.read_point_cloud(str(pc_path), format='ply')
print(pcd)
print(np.asarray(pcd.points))
if SHOW_PC:
    o3d.visualization.draw_geometries([pcd])

geometry::PointCloud with 120574 points.
[[74.536  9.937  2.752]
 [74.558 10.178  2.754]
 [74.569 10.419  2.755]
 ...
 [ 3.705 -1.394 -1.73 ]
 [ 3.611 -1.345 -1.681]
 [ 3.73  -1.377 -1.738]]


In [5]:
# visualize point cloud voxelized
downpcd, trace, old_points = pcd.voxel_down_sample_and_trace(0.05, pcd.get_min_bound(), pcd.get_max_bound(), False)
print(downpcd, pcd)
if SHOW_PC:
    o3d.visualization.draw_geometries([downpcd])

geometry::PointCloud with 91905 points. geometry::PointCloud with 120574 points.


In [6]:
downpcd_np = np.asarray(downpcd.points)
idx_sorted = np.lexsort((downpcd_np[:,1], downpcd_np[:,0]))
new_downpcd = o3d.geometry.PointCloud()
new_downpcd.points = o3d.utility.Vector3dVector(downpcd_np[idx_sorted])

In [7]:
new_downpcd

geometry::PointCloud with 91905 points.

In [8]:
# save voxalized point cloud
voxel_filename = DATA_PATH / 'open3d' / '0000000000.ply'
o3d.io.write_point_cloud(str(voxel_filename), new_downpcd, write_ascii=True)

True

In [9]:
sorted_pcd = np.asarray(pcd.points)
idx_sorted_pdc = np.lexsort((sorted_pcd[:,1], sorted_pcd[:,0]))
new_pcd = o3d.geometry.PointCloud()
new_pcd.points = o3d.utility.Vector3dVector(sorted_pcd[idx_sorted_pdc])
new_pcd

geometry::PointCloud with 120574 points.

In [10]:
pcd.points

std::vector<Eigen::Vector3d> with 120574 elements.
Use numpy.asarray() to access data.

In [11]:
# save sorted voxalized original point cloud
voxel_filename = DATA_PATH / 'kitti' / 'sorted_0000000000.ply'
o3d.io.write_point_cloud(str(voxel_filename), new_pcd, write_ascii=True)

True

In [12]:
# second point cloud
pc_path = DATA_PATH / 'kitti' / '0000000110.ply'
pcd_2 = o3d.io.read_point_cloud(str(pc_path), format='ply')
print(np.asarray(pcd_2.points))
sorted_pcd_2 = np.asarray(pcd_2.points)
idx_sorted_pdc_2 = np.lexsort((sorted_pcd_2[:,1], sorted_pcd_2[:,0]))
new_pcd_2 = o3d.geometry.PointCloud()
new_pcd_2.points = o3d.utility.Vector3dVector(sorted_pcd_2[idx_sorted_pdc_2])
# save sorted voxalized original point cloud
voxel_filename = DATA_PATH / 'kitti' / 'sorted_0000000110.ply'
o3d.io.write_point_cloud(str(voxel_filename), new_pcd_2, write_ascii=True)

[[54.454  1.736  2.051]
 [54.398  1.906  2.05 ]
 [74.238 10.491  2.744]
 ...
 [-9.549  7.993 -2.085]
 [-9.571  7.96  -2.084]
 [-9.594  7.929 -2.084]]


True

In [13]:
offset_pcd_2 = o3d.geometry.PointCloud(new_pcd_2)
offset_pcd_2.points = o3d.utility.Vector3dVector(np.asarray(offset_pcd_2.points) + [0.0, 0.0, 10.0])
o3d.visualization.draw_geometries([pcd, offset_pcd_2])

In [14]:
# get common coordinates bettwen 0 and 10
# no voxel
pcd_arr, pcd_2_arr = np.asarray(new_pcd.points), np.asarray(new_pcd_2.points)
intersec_pcd = np.array([point for point in pcd_2_arr if point in pcd_arr])
print(intersec_pcd.shape, pcd_arr.shape)
intersec_pcd

(68290, 3) (120574, 3)


array([[-78.953,  12.546,  -1.88 ],
       [-78.745,  12.767,  -1.876],
       [-78.703, -10.832,   2.018],
       ...,
       [ 78.357,  13.558,   2.019],
       [ 78.417, -13.931,   2.022],
       [ 78.581,  13.907,   2.401]])

In [15]:
intersec_pcd_pc = o3d.geometry.PointCloud()
intersec_pcd_pc.points = o3d.utility.Vector3dVector(intersec_pcd)
intersec_pcd_pc_offset = o3d.geometry.PointCloud()
intersec_pcd_pc_offset.points = o3d.utility.Vector3dVector(intersec_pcd + [0.0, 0.0, 20.0])
print(pcd, offset_pcd_2, intersec_pcd_pc_offset)
o3d.visualization.draw_geometries([pcd, offset_pcd_2, intersec_pcd_pc_offset])

geometry::PointCloud with 120574 points. geometry::PointCloud with 68526 points. geometry::PointCloud with 68290 points.
